In [1]:
import keras
import h5py
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from keras import backend as K
from tqdm import tqdm
import os

batch_size = 16
nb_epoch = 20
infodir = 'C:/Science_Research/xraynet_data/images'
imgdir = f'{infodir}/images_cat'
train_data_dir  = f'{imgdir}/train'
validation_data_dir = f'{imgdir}/test'
MODEL_NAME = 'xrayNet-keras-v2.1' #gve the model a name
# input image dimensions
img_size = 128

nb_train_samples = 100000
nb_validation_samples = 3416

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
if not os.path.exists('path_to_label_trunc.txt'):
    with open('Data_Entry_2017.csv','r') as csv:
        subjects = []
        for row in csv:
            subjects.append(row.strip().split(',')[:2])
        subjects.pop(0)
        random.shuffle(subjects)
        #subjects.sort(key=lambda x: x[1])

    normal = [case for case in subjects if 'No Finding' == case[1]]
    abnormal = [case for case in subjects if 'No Finding' != case[1]]

    normal = normal[:min(len(normal),len(abnormal))]
    abnormal = abnormal[:min(len(normal),len(abnormal))]

    trunc_lists = normal + abnormal
    random.shuffle(trunc_lists)
    with open('path_to_label_trunc.txt','w') as out_txt:
        for row in trunc_lists:
            out_txt.write(f'{imgdir}/{row[0]} {row[1]}\n')

    with open('train_paths.txt','w') as out_txt1:
        for row in trunc_lists[:-2000]:
            out_txt1.write(f'{imgdir}/{row[0]} {0 if row[1] == "No Finding" else 1}\n')

    with open('test_paths.txt','w') as out_txt2:
        for row in trunc_lists[-2000:]:
            out_txt2.write(f'{imgdir}/{row[0]} {0 if row[1] == "No Finding" else 1}\n')

In [3]:
tbCallBack = keras.callbacks.TensorBoard(log_dir=f'./TBLOGS/{MODEL_NAME}', histogram_freq=0,  
          write_graph=True, write_images=True)

In [4]:
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(512, (3, 3), input_shape=(img_size,img_size,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale')


model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples//batch_size,
        callbacks=[tbCallBack])

Found 100000 images belonging to 2 classes.
Found 3416 images belonging to 2 classes.
Epoch 1/20
  61/6250 [..............................] - ETA: 2:49:32 - loss: 0.8577 - acc: 0.43 - ETA: 1:35:16 - loss: 0.8073 - acc: 0.50 - ETA: 1:10:30 - loss: 0.8102 - acc: 0.52 - ETA: 58:06 - loss: 0.7917 - acc: 0.5312 - ETA: 50:39 - loss: 0.7797 - acc: 0.52 - ETA: 45:43 - loss: 0.7578 - acc: 0.55 - ETA: 42:12 - loss: 0.8118 - acc: 0.50 - ETA: 39:34 - loss: 0.7934 - acc: 0.51 - ETA: 37:31 - loss: 0.7789 - acc: 0.52 - ETA: 35:52 - loss: 0.7880 - acc: 0.51 - ETA: 34:32 - loss: 0.7823 - acc: 0.50 - ETA: 33:24 - loss: 0.7785 - acc: 0.50 - ETA: 32:28 - loss: 0.7851 - acc: 0.50 - ETA: 31:38 - loss: 0.7923 - acc: 0.50 - ETA: 30:55 - loss: 0.7747 - acc: 0.50 - ETA: 30:17 - loss: 0.7788 - acc: 0.50 - ETA: 29:45 - loss: 0.7764 - acc: 0.50 - ETA: 29:16 - loss: 0.7745 - acc: 0.51 - ETA: 28:51 - loss: 0.7734 - acc: 0.51 - ETA: 28:28 - loss: 0.7701 - acc: 0.52 - ETA: 28:06 - loss: 0.7625 - acc: 0.53 - ETA: 27:47

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-6d5c6cb71d0d>", line 32, in <module>
    callbacks=[tbCallBack])
  File "D:\anaconda\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\keras\models.py", line 1256, in fit_generator
    initial_epoch=initial_epoch)
  File "D:\anaconda\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 2177, in fit_generator
    class_weight=class_weight)
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 1849, in train_on_batch
    outputs = self.train_function(ins)
  File "D:\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py", line 2475, in __call__
    **self.s

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "D:\anaconda\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "D:\anaconda\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "D:\anaconda\lib\site-packages\ipykernel\kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "D:\anaconda\lib\site-packages\ipykernel\kernelbase.py", line 662, in _abort_queue
    poller.poll(50)
  File "D:\anaconda\lib\site-packages\zmq\sugar\pol

In [ ]:
model.fit(X, Y,
          epochs=epochs,
          batch_size=batch_size,
          verbose=3,
          shuffle = 'batch', 
          validation_data=(test_X,test_Y),    #data to be be used as vailidation
          callbacks=[tbCallBack]) #pushes data to tensorboard

In [ ]:
model.save('{}.{}.model'.format(MODEL_NAME,f'{batch_size}')) #save the model with the name in the working directory

In [ ]:
#print(model.predict(np.array([X[-1]]))[0])
#print(Y[-1])

pred_Y = model.predict(test_X)
pred_Y = (pred_Y > 0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_Y, pred_Y)

In [ ]:
print(cm)

### [0, 1] == Normal == 0

### [1, 0] == Abnormal == 1

### [Abnormalness, Normalness]